# Amélioration de la planification de la maintenance et de la fiabilité grâce à l’intégration de l’IA et de l’analyse décisionnelle (BI)

# CRANE SECTION

# 1. Business understanding

In [ ]:
import pandas as pd

file_path = r'/content/Data.xlsx'
df = pd.read_excel(file_path, header=4)
df.columns = df.iloc[0]
df_cleaned = df[1:].reset_index(drop=True)
df_cleaned.head()


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Sr.No,Eq status,ID: Activity/ Equipment,Description,Operation,Work center / Performed By,Outage\nType,Manpower Name,Start,End,...,Category: Activity/ Equipement,Type: Activity/ Equipement,Family: Activity/ Equipement,Week N°,Month N°,AREA,Section,Day-DSR,Year,Duration time Format hh:mm:ss
0,NaN,Available,RTG-01,"( Level 1 : 44600,375 ) / ( Level 2 : 44600,47...",NaN,NaN,INS,Mokhtar / Abdellahi,2022-02-08 09:00:00,2022-02-08 11:30:00,...,Crane EQ,Rubber Tyred Gantry Crane,RTG,W 6,février,Rubber Tyred Gantry Crane 04,CRANE,08/02/2022,2022,2:30:00
1,NaN,Available,RTG-02,( Level 1 : SPREADER ) / ( Level 2 : LANDED_SY...,NaN,NaN,BDN,Ahmed Elemine,2022-04-02 10:40:00,2022-04-02 11:20:00,...,Crane EQ,Rubber Tyred Gantry Crane,RTG,W 13,avril,Rubber Tyred Gantry Crane 05,CRANE,2022-04-02 00:00:00,2022,0:40:00
2,NaN,Available,STS-02,( Level 1 : POWER_AND_CONTROL ) / ( Level 2 : ...,NaN,NaN,CM,Mohamed Mamoun,2022-04-04 07:30:00,2022-04-04 07:40:00,...,Crane EQ,Ship To Shore,STS,W 14,avril,Crane STS 02,CRANE,04/04/2022,2022,0:10:00
3,NaN,Available,STS-01,( Level 1 : POWER_AND_CONTROL ) / ( Level 2 : ...,NaN,NaN,CM,Mohamed Habib,2022-04-04 07:30:00,2022-04-04 07:40:00,...,Crane EQ,Ship To Shore,STS,W 14,avril,Crane STS 02,CRANE,04/04/2022,2022,0:10:00
4,NaN,Available,STS-02,( Level 1 : POWER_AND_CONTROL ) / ( Level 2 : ...,NaN,NaN,CMP,Mohamed Mamoun,2022-04-04 07:30:00,2022-04-04 07:40:00,...,Crane EQ,Ship To Shore,STS,W 14,avril,Crane STS 02,CRANE,04/04/2022,2022,0:10:00


In [ ]:
df_cleaned.dtypes

,0
0,
Sr.No,object
Eq status,object
ID: Activity/ Equipment,object
Description,object
Operation,object
Work center / Performed By,object
Outage\nType,object
Manpower Name,object
Start,object


In [ ]:

df_cleaned['Duration time Format hh:mm:ss'] = pd.to_timedelta(df_cleaned['Duration time Format hh:mm:ss'], errors='coerce')
df_cleaned['Duration in Minutes'] = df_cleaned['Duration time Format hh:mm:ss'].dt.total_seconds() / 60


In [ ]:
rows, cols = df_cleaned.shape
print(f"📊 Nombre de lignes : {rows}")
print(f"📊 Nombre de colonnes : {cols}")


📊 Nombre de lignes : 13709
📊 Nombre de colonnes : 30


In [ ]:
df_cleaned = df_cleaned.drop(columns=["Sr.No"])
df_cleaned .columns

Index(['Eq status', 'ID: Activity/ Equipment ', 'Description', 'Operation ',
       'Work center / Performed By', 'Outage\nType', 'Manpower Name', 'Start',
       'End', 'Duration', 'No. of Manpower', 'Total \nMan Hours', 'Status',
       'WO / Notification N°', 'Supervisor', 'Shift', 'WCR', 'Completion',
       'Category: Activity/ Equipement', 'Type: Activity/ Equipement',
       'Family: Activity/ Equipement', 'Week N°', 'Month N°', 'AREA',
       'Section', 'Day-DSR', 'Year', 'Duration time Format hh:mm:ss',
       'Duration in Minutes'],
      dtype='object', name=0)

In [ ]:
nan_percentage = (df_cleaned.isna().sum() / len(df_cleaned)) * 100
print(nan_percentage)


0
Eq status                          37.216427
ID: Activity/ Equipment            42.679991
Description                         0.255307
Operation                          32.526078
Work center / Performed By         21.868845
Outage\nType                        0.554380
Manpower Name                       0.415785
Start                               0.000000
End                                 0.094828
Duration                            0.094828
No. of Manpower                     0.488730
Total \nMan Hours                   0.547086
Status                              0.853454
WO / Notification N°               35.990955
Supervisor                         15.303815
Shift                              15.799840
WCR                               100.000000
Completion                         99.985411
Category: Activity/ Equipement      0.000000
Type: Activity/ Equipement          0.000000
Family: Activity/ Equipement        0.000000
Week N°                             0.000000
Month N°

#2.Data processing

In [ ]:
colonnes_a_supprimer = [
    "End",
    "Duration",
    "No. of Manpower",
    "Total \nMan Hours",
    "Status",
    "Description",
    "Outage\nType",
    "Manpower Name"
]

df_cleaned = df_cleaned.dropna(subset=colonnes_a_supprimer)

print(df_cleaned.isna().sum())


0
Eq status                          4949
ID: Activity/ Equipment            5757
Description                           0
Operation                          4213
Work center / Performed By         2784
Outage\nType                          0
Manpower Name                         0
Start                                 0
End                                   0
Duration                              0
No. of Manpower                       0
Total \nMan Hours                     0
Status                                0
WO / Notification N°               4754
Supervisor                         1882
Shift                              1952
WCR                               13420
Completion                        13420
Category: Activity/ Equipement        0
Type: Activity/ Equipement            0
Family: Activity/ Equipement          0
Week N°                               0
Month N°                              0
AREA                                  0
Section                               

In [ ]:
len(df_cleaned)

13420

In [ ]:

print(" Nombre de valeurs uniques par colonne :\n")
for col in df_cleaned.columns:
    valeurs_uniques = df_cleaned[col].nunique(dropna=True)
    print(f"✅ {col}: {valeurs_uniques} valeurs uniques")

 Nombre de valeurs uniques par colonne :

✅ Eq status: 13 valeurs uniques
✅ ID: Activity/ Equipment : 42 valeurs uniques
✅ Description: 4616 valeurs uniques
✅ Operation : 3043 valeurs uniques
✅ Work center / Performed By: 5 valeurs uniques
✅ Outage
Type: 32 valeurs uniques
✅ Manpower Name: 2756 valeurs uniques
✅ Start: 8177 valeurs uniques
✅ End: 8422 valeurs uniques
✅ Duration: 313 valeurs uniques
✅ No. of Manpower: 11 valeurs uniques
✅ Total 
Man Hours: 553 valeurs uniques
✅ Status: 9 valeurs uniques
✅ WO / Notification N°: 4440 valeurs uniques
✅ Supervisor: 24 valeurs uniques
✅ Shift: 6 valeurs uniques
✅ WCR: 0 valeurs uniques
✅ Completion: 0 valeurs uniques
✅ Category: Activity/ Equipement: 5 valeurs uniques
✅ Type: Activity/ Equipement: 10 valeurs uniques
✅ Family: Activity/ Equipement: 12 valeurs uniques
✅ Week N°: 52 valeurs uniques
✅ Month N°: 12 valeurs uniques
✅ AREA: 19 valeurs uniques
✅ Section: 4 valeurs uniques
✅ Day-DSR: 1044 valeurs uniques
✅ Year: 4 valeurs uniques
✅ D

In [ ]:
print("Eq status:", df_cleaned["Eq status"].nunique(), "valeurs uniques")
print(df_cleaned["Eq status"].unique())


Eq status: 13 valeurs uniques
['Available' nan 'Out of order' 'Out Of Order' 'available' 'out of order'
 'Available ' 'out of order ' 'Out of Order' 'Not Available'
 'Out of order ' 'Maintenance' 'Support' 'Out Of Service']


In [ ]:
mapping = {
    'available': 'Available',
    'available ': 'Available',
    'out of order': 'Out Of Order',
    'out of order ': 'Out Of Order',
    'Out of Order': 'Out Of Order',
    'Out Of Order': 'Out Of Order',
    'Out of order ': 'Out Of Order',
    'Out Of Service': 'Out Of Service',


}
df_cleaned["Eq status"] = df_cleaned["Eq status"].str.strip().str.lower().replace(mapping)

df_cleaned["Eq status"] = df_cleaned["Eq status"].str.title()

print("Eq status:", df_cleaned["Eq status"].nunique(), "valeurs uniques")
print(df_cleaned["Eq status"].unique())


Eq status: 6 valeurs uniques
['Available' nan 'Out Of Order' 'Not Available' 'Maintenance' 'Support'
 'Out Of Service']


<ipython-input-11-cb8de459d8a8>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["Eq status"] = df_cleaned["Eq status"].str.strip().str.lower().replace(mapping)
<ipython-input-11-cb8de459d8a8>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["Eq status"] = df_cleaned["Eq status"].str.title()


In [ ]:
out_of_order_count = df_cleaned["Eq status"].value_counts().get("Out Of Order", 0)
out_of_service_count = df_cleaned["Eq status"].value_counts().get("Out Of Service", 0)

pourcentage_Out_Of_Order = (out_of_order_count / len(df_cleaned["Eq status"])) * 100
pourcentage_out_of_service = (out_of_service_count / len(df_cleaned["Eq status"])) * 100

print(f"Pourcentage de 'Out Of Order' : {pourcentage_Out_Of_Order:.2f}%")
print(f"Pourcentage de 'Out Of Service' : {pourcentage_out_of_service:.2f}%")


Pourcentage de 'Out Of Order' : 17.79%
Pourcentage de 'Out Of Service' : 0.31%


In [ ]:
import numpy as np
df_cleaned["Eq status"] = df_cleaned["Eq status"].str.strip()
mask = df_cleaned["Eq status"] == "Not Available"
freq = df_cleaned["Eq status"].value_counts(normalize=True)
out_of_order_pct = 17.79 / 100
out_of_service_pct = 0.31 / 100
total_pct = out_of_order_pct + out_of_service_pct
p = [out_of_order_pct / total_pct, out_of_service_pct / total_pct]
df_cleaned.loc[mask, "Eq status"] = np.random.choice(["Out Of Order", "Out Of Service"], size=mask.sum(), p=p)
print("Valeurs uniques finales :", df_cleaned["Eq status"].unique())
print("Nombre d'occurrences après remplacement :\n", df_cleaned["Eq status"].value_counts())


Valeurs uniques finales : ['Available' nan 'Out Of Order' 'Out Of Service' 'Maintenance' 'Support']
Nombre d'occurrences après remplacement :
 Eq status
Available         5943
Out Of Order      2477
Out Of Service      43
Maintenance          5
Support              3
Name: count, dtype: int64


<ipython-input-13-5f7531cbee1f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["Eq status"] = df_cleaned["Eq status"].str.strip()


In [ ]:
df_cleaned = df_cleaned[~df_cleaned["Eq status"].isin(["Maintenance", "Support"])]

print("Valeurs uniques après suppression :", df_cleaned["Eq status"].unique())
print("Nombre d'occurrences après suppression :\n", df_cleaned["Eq status"].value_counts())


Valeurs uniques après suppression : ['Available' nan 'Out Of Order' 'Out Of Service']
Nombre d'occurrences après suppression :
 Eq status
Available         5943
Out Of Order      2477
Out Of Service      43
Name: count, dtype: int64


In [ ]:
nan_count = df_cleaned["Eq status"].isna().sum()
print("Nombre de NaN dans 'Eq status' :", nan_count)


Nombre de NaN dans 'Eq status' : 4949


Imputation colonne Eq status

In [ ]:
print("status:", df_cleaned['Status'].nunique(), "valeurs uniques")
print("Valeurs uniques dans 'status':", df_cleaned['Status'].unique())


status: 9 valeurs uniques
Valeurs uniques dans 'status': ['Closed' 'Active' 'ACtive' 'Closed ' 'Canceled' 'closed' 'closed '
 'Active ' 'CLOSED']


In [ ]:
df_cleaned['Status'] = df_cleaned['Status'].replace({
    'closed': 'Closed',
    'closed ': 'Closed',
    'CLOSED': 'Closed',
    'Active': 'Active',
    'ACtive': 'Active',
    'active': 'Active',
    'Closed ': 'Closed',
    'Active ' : 'Active'
})

In [ ]:
print("status:", df_cleaned['Status'].nunique(), "valeurs uniques")
print("Valeurs uniques dans 'status':", df_cleaned['Status'].unique())


status: 3 valeurs uniques
Valeurs uniques dans 'status': ['Closed' 'Active' 'Canceled']


In [ ]:
print("ID: Activity/ Equipment :", df_cleaned["ID: Activity/ Equipment "].nunique(), "valeurs uniques")
print(df_cleaned["ID: Activity/ Equipment "].unique())


ID: Activity/ Equipment : 42 valeurs uniques
['RTG-01' 'RTG-02' 'STS-02' 'STS-01' 'RTG-04' 'RTG-03' nan 'RTG'
 'STS / RTG' 'RSP-01' 'RSP-02' 'RSP-03' 'PST-04' '--------' ' ' 'SSP03-B'
 'SSP04-Z' 'SSP01-B' 'SSP02-B' 'STS' 'Substation ' 'CCB' 'ECH' 'TT'
 'RTGDG-01' 'RTG-02 - RTG-04' 'RSP-05' 'RSP-04' 'Quay Area' 'Yard Area'
 'Quay area' 'Yard area' 'yard area' 'sts' 'Quay area ' 'EDG' 'Yard area '
 'OHF-01' 'GENERAL' 'General ' 'RTG-01, RTG-02, RTG-03, RTG-04'
 'STS-01, STS-02' 'Other']


In [ ]:
df_cleaned["ID: Activity/ Equipment "] = df_cleaned["ID: Activity/ Equipment "].replace({
    'Quay Area': 'QUAY AREA',
    'Yard Area': 'YARD AREA',
    'Quay area': 'QUAY AREA',
    'Yard area': 'YARD AREA',
    'yard area': 'YARD AREA',
    'Quay area ': 'QUAY AREA',
    'Yard area ': 'YARD AREA',
    'Substation ': 'SUBSTATION',
    'General ': 'GENERAL',
})
df_cleaned["ID: Activity/ Equipment "] = df_cleaned["ID: Activity/ Equipment "].str.strip()

df_cleaned["ID: Activity/ Equipment "] = df_cleaned["ID: Activity/ Equipment "].replace("--------", None)
df_cleaned = df_cleaned[df_cleaned["ID: Activity/ Equipment "] != "--------"]

print("ID: Activity/ Equipment :", df_cleaned["ID: Activity/ Equipment "].nunique(), "valeurs uniques")
print(df_cleaned["ID: Activity/ Equipment "].unique())


ID: Activity/ Equipment : 35 valeurs uniques
['RTG-01' 'RTG-02' 'STS-02' 'STS-01' 'RTG-04' 'RTG-03' nan 'RTG'
 'STS / RTG' 'RSP-01' 'RSP-02' 'RSP-03' 'PST-04' None '' 'SSP03-B'
 'SSP04-Z' 'SSP01-B' 'SSP02-B' 'STS' 'SUBSTATION' 'CCB' 'ECH' 'TT'
 'RTGDG-01' 'RTG-02 - RTG-04' 'RSP-05' 'RSP-04' 'QUAY AREA' 'YARD AREA'
 'sts' 'EDG' 'OHF-01' 'GENERAL' 'RTG-01, RTG-02, RTG-03, RTG-04'
 'STS-01, STS-02' 'Other']


In [ ]:
nan_count = df_cleaned["ID: Activity/ Equipment "].isna().sum()
print("Nombre de NaN dans ' ID: Activity/ Equipment ' :", nan_count)


Nombre de NaN dans ' ID: Activity/ Equipment ' : 5766


Imputation de ID: Activity/ Equipment

In [ ]:
print("Work center / Performed By:", df_cleaned["Work center / Performed By"].nunique(), "valeurs uniques")
print(df_cleaned["Work center / Performed By"].unique())


Work center / Performed By: 5 valeurs uniques
[nan 'Electrical' 'Mechanical' 'mechanical' 'ELECTRICAL' 'electrical']


In [ ]:

df_cleaned["Work center / Performed By"] = df_cleaned["Work center / Performed By"].replace({
    'electrical': 'Electrical',
    'mechanical': 'Mechanical',
    'ELECTRICAL': 'Electrical',
    'MECHANICAL': 'Mechanical'
})

print("Work center / Performed By:", df_cleaned["Work center / Performed By"].nunique(), "valeurs uniques")
print(df_cleaned["Work center / Performed By"].unique())


Work center / Performed By: 2 valeurs uniques
[nan 'Electrical' 'Mechanical']


In [ ]:
nan_count = df_cleaned["Work center / Performed By"].isna().sum()
print("Nombre de NaN dans 'Work center / Performed By:' :", nan_count)


Nombre de NaN dans 'Work center / Performed By:' : 2784


Imputation de Work center / Performed By

In [ ]:
df_cleaned["Supervisor"] = df_cleaned["Supervisor"].str.strip()
df_cleaned["Supervisor"] = df_cleaned["Supervisor"].replace({
    'Mohamed Vadel Bounenna ISSA ': 'Mohamed Vadel Bounenna ISSA',
    'Mohamed Habib Ellah Sidi Mohamed ': 'Mohamed Habib Ellah Sidi Mohamed',
    'Ahmed Elemine BAH ': 'Ahmed Elemine Bah',
    'Ahmed Elemine Bah': 'Ahmed Elemine BAH',
    'Mohamed Mezid ': 'Mohamed Mezid',
    'Mohamed Mamoun Bay ': 'Mohamed Mamoun Bay',
})

print("Supervisor:", df_cleaned["Supervisor"].nunique(), "valeurs uniques")
print(df_cleaned["Supervisor"].unique())


Supervisor: 16 valeurs uniques
[nan 'Mohamed Vadel Bounenna ISSA' 'Mohamed Habib Ellah Sidi Mohamed'
 'Ahmed Elemine BAH' 'Ahmed Vall Lekoueiry'
 'Ahmed Saleck Mohameden El Vadhel' 'Touhana Ahmed Sidi Ahmed'
 'Mohamed Mezid' 'Mohamed Mamoun Bay' 'Mohamed MEZID' 'Mohamed Mamoun bay'
 'Mohamed Vadel Bounenna Issa' '100042 Mohamed Vadel Bounenna ISSA'
 '100359 Mohamed Lemine MEZID' '100078 Ahmed Elemine Bah'
 '100077 Mohamed Habib Ellah Sidi Mohamed' '100087 Mohamed Mamoun Bay']


In [ ]:
nan_count = df_cleaned["Supervisor"].isna().sum()
print("Nombre de NaN dans 'Supervisor' :", nan_count)

Nombre de NaN dans 'Supervisor' : 1882


Imputtion de Supervisor

In [ ]:

print("Shift", df_cleaned["Shift"].nunique(), "valeurs uniques")
print(df_cleaned["Shift"].unique())


Shift 6 valeurs uniques
[nan 'A' 'B' 'N' 'D' 'C' 'E']


In [ ]:
nan_count = df_cleaned["Shift"].isna().sum()
print("Nombre de NaN dans 'Shift' :", nan_count)

Nombre de NaN dans 'Shift' : 1952


Imputation de shift